<a href="https://colab.research.google.com/github/GuanRuLai/Python-Deep-Learning/blob/main/GAN(generate_numbers).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data preprocessing

## Import dataset & Split training and testing set

In [ ]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np

from keras.datasets import mnist
from keras.layers import Dense, Flatten, Reshape, LeakyReLU
from keras.models import Sequential
from keras.datasets import mnist

(X_train, _), (_, _) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


## Rescale -1 to 1 to fit tanh() activation function of output layer

In [ ]:
X_train = (X_train.astype(np.float32) - 127.5) / 127.5

## Add a color channel dimension

In [ ]:
X_train =  np.expand_dims(X_train, axis=3)

# Neural network processing

## Set the hyperparameters

In [ ]:
# set the shape of MNIST
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

# set the length of noise for GANs generator input layer
z_dim = 100

epochs = 10000
batch_size = 128

# set the times of training that shows acc / loss once
sample_interval = 100

## Define generator

In [ ]:
def build_generator(img_shape, z_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=z_dim))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    return model

## Define discriminator

In [ ]:
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(1, activation='sigmoid'))
    return model

## Define GAN: combine generator and discriminator

In [ ]:
def build_gan(generator, discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

## Create and compile discriminator

In [ ]:
discriminator = build_discriminator(img_shape)
discriminator.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

## Freeze discriminator's weights when training generator

In [ ]:
discriminator.trainable = False

## Create generator

In [ ]:
generator = build_generator(img_shape, z_dim)

## Create and compile GAN

In [ ]:
gan = build_gan(generator, discriminator)
gan.compile(optimizer="adam", loss='binary_crossentropy')

## Define the function of showing graphs during training process

In [ ]:
def sample_images(generator, image_grid_rows=4, image_grid_columns=4):
    # sample random noise
    noise = np.random.normal(loc=0, scale=1, size=(image_grid_rows * image_grid_columns, z_dim))

    # generate images from random noise
    gen_imgs = generator.predict(noise)

    # rescale image pixel values to [0, 1]
    gen_imgs = 0.5 * gen_imgs + 0.5

    # Set image grid
    fig, axs = plt.subplots(image_grid_rows,
                            image_grid_columns,
                            figsize=(4, 4),
                            sharey=True,
                            sharex=True)
    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            # Output a grid of images
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1

    plt.show()

## Define training function

In [ ]:
losses = []
accuracies = []
iteration_checkpoints = []

def train(iterations, batch_size, sample_interval):
    # labels for real images: all ones
    real = np.ones(batch_size, 1) # 128 rows, 1 column

    # labels for fake images: all zeros
    fake = np.zeros(batch_size, 1) # 128 rows, 1 column

    for iteration in range(iterations):

        # turn off the stdout temporarily
        # to supress 4/4 [=====================] - 0s 3ms/step
        original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

        # ---- train discriminator ----

        # get a random batch(128) of real images
        idx = np.random.randint(0, X_train.shape[0], batch_size) # get 128 columns from index 0~59999
        imgs = X_train[idx]

        # generate a batch of fake images
        noise = np.random.normal(0, 1, (batch_size, z_dim)) # mean=0, std=1, 128 rows, 100 columns
        gen_imgs = generator.predict(noise)

        # train discriminator
        d_loss_real = discriminator.train_on_batch(imgs, real)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---- train generator ----

        # generate a batch of fake images
        noise = np.random.normal(0, 1, (batch_size, z_dim)) # mean=0, std=1, 128 rows, 100 columns

        # set the noise as True(1) in purpose
        g_loss = gan.train_on_batch(noise, real)

        # turn stdout back on
        sys.stdout.close()
        sys.stdout = original_stdout

        if (iteration + 1) % sample_interval == 0:

            # save losses and accuracies so they can be plotted after training
            losses.append((d_loss, g_loss))
            accuracies.append(100.0 * accuracy)
            iteration_checkpoints.append(iteration + 1)

            # output training progress
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %
                (iteration + 1, d_loss, 100.0 * accuracy, g_loss))

            # output a sample of generated image
            sample_images(generator)

## Model training

In [ ]:
train(iterations=epochs, batch_size=batch_size, sample_interval=sample_interval)